In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch

# wybór urządzenia
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# nazwa modelu
model_name = "gpt2-medium"

# tokenizer i model
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name).to(device)
model.eval()


In [ ]:
# prompt
prompt = "Once upon a time in a distant galaxy"

# tokenizacja + przeniesienie na GPU
inputs = tokenizer(prompt, return_tensors="pt")
inputs = {k: v.to(device) for k, v in inputs.items()}

# generowanie tekstu
with torch.no_grad():
    output = model.generate(
        **inputs,
        max_length=100,
        do_sample=True,
        temperature=0.8,
        top_k=50,
        top_p=0.95
    )

# dekodowanie do tekstu
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(generated_text)


In [ ]:
def sentence_probability(model, tokenizer, text, device):
    """
    Zwraca:
    - log_prob: log P(zdanie)
    - prob: P(zdanie) (uwaga: bardzo mała liczba)
    """

    # tokenizacja
    inputs = tokenizer(text, return_tensors="pt")
    input_ids = inputs["input_ids"].to(device)

    with torch.no_grad():
        outputs = model(input_ids, labels=input_ids)
        # loss = średni NLL na token
        loss = outputs.loss

    # liczba tokenów (bez pierwszego — nie jest przewidywany)
    num_tokens = input_ids.size(1) - 1

    # log P(zdanie)
    log_prob = -loss.item() * num_tokens

    return log_prob


In [ ]:
text = "Once upon a time in a distant galaxy"

log_p = sentence_probability(model, tokenizer, text, device)

print(f"log P(sentence) = {log_p:.4f}")
